# Sevilla + cloro

In [4]:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map, TileLayer
from folium.plugins import HeatMap, MarkerCluster
import geopandas as gpd
import pandas as pd
import json
import ast

## cargamos el .shp

In [7]:
sevilla_barrios_gdf = gpd.read_file("DATA/sevilla_barrios_forma/zipfolder")

In [189]:
#sevilla_barrios_gdf.crs

In [198]:
sevilla_barrios_gdf.head()

,BARRIO,BARRIOS,DISTRITO,AREA_M2,SHAPE_AREA,SHAPE_LEN,geometry
0,"Tiro de Línea, Santa Genoveva","TIRO DE LINEA, SANTA GENOVEVA",Sur,284445,0.0,0.0,"POLYGON ((236897.329 4139935.844, 236854.159 4..."
1,"Felipe II, Los Diez Mandamientos","FELIPE II, LOS DIEZ MANDAMIENTOS",Sur,102787,0.0,0.0,"POLYGON ((237015.891 4140043.367, 236937.243 4..."
2,La Plata,LA PLATA,Cerro - Amate,595268,0.0,0.0,"POLYGON ((239079.232 4140474.754, 239203.005 4..."
3,El Porvenir,EL PORVENIR,Sur,335062,0.0,0.0,"POLYGON ((236254.663 4140320.159, 236254.318 4..."
4,Tablada,TABLADA,Los Remedios,13176347,0.0,0.0,"POLYGON ((234984.542 4139970.508, 234985.619 4..."


## podemos pintar simplemente los barrios:

In [195]:
sevilla_map_barr = folium.Map(location=[37.39161,-5.97640], zoom_start=12, tiles="cartodbpositron");
folium.GeoJson(data=sevilla_barrios_gdf["geometry"]).add_to(sevilla_map_barr);
sevilla_map_barr

## pasamos de geoDF a json:

In [17]:
sevilla_barrios_json = ast.literal_eval(sevilla_barrios_gdf.to_json())

In [19]:
type(sevilla_barrios_json)

dict

In [20]:
sevilla_barrios_json.keys()

dict_keys(['type', 'features'])

In [22]:
len(sevilla_barrios_json["features"])

108

In [24]:
sevilla_barrios_json["features"][5]["properties"]

{'AREA_M2': 815317,
 'BARRIO': 'El Juncal, Híspalis',
 'BARRIOS': 'EL JUNCAL, HISPALIS',
 'DISTRITO': 'Sur',
 'SHAPE_AREA': 0.0,
 'SHAPE_LEN': 0.0}

## (también podemos guardar el json y cargarlo) :

In [207]:
sevilla_barrios_gdf.to_file("DATA/sevibarr.geojson", driver="GeoJSON") 

In [208]:
import json
geo_json = r"DATA/sevibarr.geojson"

In [209]:
with open (geo_json) as geo_file:
    sevi_barrios = json.load(geo_file)

In [30]:
type(sevi_barrios)

dict

In [31]:
sevi_barrios.keys()

dict_keys(['type', 'crs', 'features'])

In [32]:
len(sevi_barrios["features"])

108

In [216]:
sevi_barrios["features"][6]["geometry"]

{'type': 'Polygon',
 'coordinates': [[[236204.24580532667, 4138225.571836541],
   [236285.9292156725, 4138043.2243276895],
   [236306.55652409818, 4137990.649677068],
   [236317.45724143335, 4137962.8654732537],
   [236321.363468031, 4137946.5433317684],
   [236331.43830487927, 4137904.44519864],
   [236336.81539078683, 4137881.976754628],
   [236349.53247389305, 4137744.759136137],
   [236354.824681226, 4137688.711308089],
   [236364.79704015155, 4137603.1913719326],
   [236366.5649137483, 4137588.0308314706],
   [236368.74534159288, 4137580.7207430284],
   [236374.82646139938, 4137560.331254223],
   [236405.68375297327, 4137482.991955436],
   [236418.08138604646, 4137451.918598123],
   [236518.69476204825, 4137229.5378494263],
   [236571.93718378138, 4137111.858467374],
   [236579.45716337184, 4137114.9983334006],
   [236809.40388722986, 4136587.2457064474],
   [236871.0985401484, 4136357.9473065003],
   [236864.2993916126, 4136356.711597387],
   [236723.42965832405, 4136331.10727968

In [36]:
sevi_barrios["features"][5]["properties"]

{'BARRIO': 'El Juncal, Híspalis',
 'BARRIOS': 'EL JUNCAL, HISPALIS',
 'DISTRITO': 'Sur',
 'AREA_M2': 815317,
 'SHAPE_AREA': 0.0,
 'SHAPE_LEN': 0.0}

In [194]:
listajson=[elem["properties"]["BARRIOS"] for elem in sevi_barrios["features"]]
listajson=sorted(listajson)

In [217]:
sevilla_map_barr = folium.Map(location=[37.39161,-5.97640], zoom_start=12, tiles="cartodbpositron");
folium.GeoJson(data=sevi_barrios["features"][8]["geometry"]).add_to(sevilla_map_barr);
sevilla_map_barr

## cargamos el data:

In [176]:
df = pd.read_csv("DATA/sev_pobl.csv", index_col=0, encoding="latin-1")
df["HAB"]=df.HOMBRES+df.MUJERES
df.drop(columns=["SECCION"],inplace=True)
df

,BARRIO,EDAD,HOMBRES,MUJERES,HAB
DISTRITO,,,,,
1,SANTA CATALINA,00,7,8,15
1,SANTA CATALINA,01,4,5,9
1,SANTA CATALINA,02,6,3,9
1,SANTA CATALINA,03,2,8,10
1,SANTA CATALINA,04,7,12,19
...,...,...,...,...,...
11,LOS REMEDIOS,86,2,5,7
11,LOS REMEDIOS,87,1,3,4
11,LOS REMEDIOS,88,1,5,6


In [40]:
len(df.BARRIO.unique())

108

In [192]:
listadata=sorted(df.BARRIO.unique())

In [190]:
dicci={}
for i in range(108):
    dicci[f"{listadata[i]}"]=f"{listajson[i]}"

In [175]:
df

,SECCION,BARRIO,EDAD,HOMBRES,MUJERES,HAB
DISTRITO,,,,,,
1,1,SANTA CATALINA,00,7,8,15
1,1,SANTA CATALINA,01,4,5,9
1,1,SANTA CATALINA,02,6,3,9
1,1,SANTA CATALINA,03,2,8,10
1,1,SANTA CATALINA,04,7,12,19
...,...,...,...,...,...,...
11,19,LOS REMEDIOS,86,2,5,7
11,19,LOS REMEDIOS,87,1,3,4
11,19,LOS REMEDIOS,88,1,5,6


In [177]:
df=df.replace(dicci)

In [178]:
listadatabien=sorted(df.BARRIO.unique())
listadatabien

['AEROPUERTO VIEJO',
 'ALFALFA',
 'AMATE',
 'ARBOL GORDO',
 'ARENAL',
 'AVD. DE LA PAZ',
 'BAMI',
 'BARRIO LEON',
 'BDA. DE PINEDA',
 'BDA. PINO MONTANO',
 'BEGOÑA, SANTA CATALINA',
 'BELLAVISTA',
 'CAMPOS DE SORIA',
 'CARRETERA DE CARMONA, MARIA AUXILIADORA, FONTANAL',
 'CISNEO ALTO, SANTA MARIA DE GRACIA',
 'CIUDAD JARDIN',
 'COLORES, ENTREPARQUES',
 'CONSOLACION',
 'CRUZ ROJA, CAPUCHINOS',
 'DOCTOR BARRAQUER, G. RENFE, POLICLINICO',
 'EL CANO, LOS BERMEJALES',
 'EL CARMEN',
 'EL CERRO',
 'EL GORDILLO',
 'EL JUNCAL, HISPALIS',
 'EL PLANTINAR',
 'EL PORVENIR',
 'EL ROCIO',
 'EL TARDON, EL CARMEN',
 'EL TORREJON, EL CEREZO',
 'ENCARNACION, REGINA',
 'FELIPE II, LOS DIEZ MANDAMIENTOS',
 'FERIA',
 'GIRALDA SUR',
 'HELIOPOLIS',
 'HERMANDADES, LA CARRASCA',
 'HUERTA DE LA SALUD',
 'HUERTA DE SANTA TERESA',
 'HUERTA DEL PILAR',
 'ISLA DE GARZA',
 'JUAN XXIII',
 'LA BACHILLERA',
 'LA BARZOLA',
 'LA BUHAIRA',
 'LA CALZADA',
 'LA CORZA',
 'LA FLORIDA',
 'LA OLIVA',
 'LA PALMILLA, DOCTOR MARAÑO

In [179]:
check=[listadatabien[i]==listajson[i] for i in range(108)]

In [180]:
check

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

## agrupamos por barrio :

In [184]:
dfg = df.groupby(df.BARRIO, sort = False).sum()
dfg["Barrio"]=dfg.index
dfg

,HOMBRES,MUJERES,HAB,Barrio
BARRIO,,,,
"SANTA AURELIA, CANTABRICO, ATLANTICO, LA ROMERIA",2103,2353,4456,"SANTA AURELIA, CANTABRICO, ATLANTICO, LA ROMERIA"
HELIOPOLIS,3298,3367,6665,HELIOPOLIS
FERIA,1850,2427,4277,FERIA
SAN JOSE OBRERO,2891,3263,6154,SAN JOSE OBRERO
SAN ROQUE,2706,2986,5692,SAN ROQUE
...,...,...,...,...
"PEDRO SALVADOR, LAS PALMERITAS",1686,1850,3536,"PEDRO SALVADOR, LAS PALMERITAS"
BARRIO LEON,396,460,856,BARRIO LEON
"FELIPE II, LOS DIEZ MANDAMIENTOS",6794,7213,14007,"FELIPE II, LOS DIEZ MANDAMIENTOS"


In [196]:
sevi_barrios

{'type': 'FeatureCollection',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:EPSG::25830'}},
 'features': [{'type': 'Feature',
   'properties': {'BARRIO': 'Tiro de Línea, Santa Genoveva',
    'BARRIOS': 'TIRO DE LINEA, SANTA GENOVEVA',
    'DISTRITO': 'Sur',
    'AREA_M2': 284445,
    'SHAPE_AREA': 0.0,
    'SHAPE_LEN': 0.0},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[236897.32864036903, 4139935.8439925164],
      [236854.15863332018, 4139892.6584467855],
      [236821.25568745282, 4139848.1062162393],
      [236786.74750310284, 4139790.910434604],
      [236763.54111853862, 4139739.74822137],
      [236684.20018045855, 4139729.82844621],
      [236675.81526145138, 4139729.324544499],
      [236683.83276779132, 4139669.76535288],
      [236518.30212701607, 4139647.9085094035],
      [236513.4985780177, 4139667.6101992023],
      [236345.90258274757, 4139646.86575621],
      [236338.68798076108, 4139672.5597625137],
      [236331.09422389124, 4139710.17

## Creamos el mapa y el coroplético:

In [203]:
algo = folium.GeoJson(data=sevilla_barrios_gdf[["geometry", "BARRIOS"]])

In [205]:
print(algo)

In [211]:
sevilla_clo = folium.Map(location=[37.39161,-5.97640], zoom_start=12, tiles="cartodbpositron");

sevilla_clo.choropleth(
    geo_data = sevi_barrios,
    data = dfg, 
    columns = ["Barrio","HAB"],
    key_on = "feature.properties.BARRIOS", 
    fill_color = "YlGnBu",
    fill_opacity = 0.4,
    line_opacity = 1,
    legend_name = "Población total por barrios",
    smooth_factor = 1,
    highlight=True
);

In [212]:
sevilla_clo